In [ ]:
import sys
print(sys.version)

In [ ]:
import mlpy

In [ ]:
from alefuncs import *

In [ ]:
s = 'ATGGTGCTTGTGGGCACGCAGGATGCCATCAGCGCTGCGAATCCCCGGGTTATCGACGACAGCAGAGCCCGCAAGCTCTCCACAGATCTGAAGCGGTGCACCTACTATGAGACGTGCGCGACCTACGGGCTCAATGTGGAGCGTGTCTTCCAGGACGTGGCCCAGAAGGTAGTGGCCTTGCGAAAGAAGCAGCAACTGGCCATCGGGCCCTGCAAGTCACTGCCCAACTCGCCCAGCCACTCGGCCGTGTCCGCCGCCTCCATCCCGGCCGTGCACATCAACCAGGCCACGAATGGCGGCGGCAGCGCCTTCAGCGACTACTCGTCCTCAGTCCCCTCCACCCCCAGCATCAGCCAGCGGGAGCTGCGCATCGAGACCATCGCTGCCTCCTCCACCCCCACACCCATCCGAAAGCAGTCCAAGCGGCGCTCCAACATCTTCACGGACTTGATTAGAGACCAAGGATTTCGTGGTGATGGAGGATCAACCACAGGTTTGTCTGCTACCCCCCCTGCCTCATTACCTGGCTCACTAACTAACGTGAAAGCCTTACAGAAATCTCCAGGACCTCAGCGAGAAAGGAAGTCATCTTCATCCTCAGAAGACAGGAATCGAATGAAAACACTTGGTAGACGGGACTCGAGTGATGATTGGGAGATTCCTGATGGGCAG'
gc_content(s, percent=True)

In [ ]:
from subprocess import run
import pexpect
from getpass import getpass

def ssl_encrypt(infile, outfile):
    '''(file_path, file_path) => encrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter encryption pwd:')
    if getpass('repeat pwd:') == pwd:
        run(f'openssl enc -aes-256-cbc -a -salt -pass pass:{pwd} -in {infile} -out {outfile}',shell=True)
    else:
        print("passwords don't match.")

    
def ssl_decrypt(infile, outfile):
    '''(file_path, file_path) => decrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter decryption pwd:')
    run(f'openssl enc -d -aes-256-cbc -a -pass pass:{pwd} -in {infile} -out {outfile}', shell=True)

In [ ]:
infile = 'alefuncs.py'
outfile = 'alefuncs.enc'
print(f'openssl aes-256-cbc -a -salt -in {infile} -out {outfile} -base64')

infile = 'alefuncs.enc'
outfile = 'alefuncs.dec.py'
print(f'openssl enc -d -aes-256-cbc -in {infile} -out {outfile}')

In [ ]:
ssl_encrypt('alefuncs.py','alefuncs.enc')
ssl_decrypt('alefuncs.enc', 'a2.py')

In [ ]:
run(f'openssl enc -aes-256-cbc -pass  -p -in alefuncs.py -out alefuncs.enc -base64',shell=True)


In [ ]:
import socket   #for sockets
import time

class Server:
    def __init__(self):
        self.id = 'server.007'
        self.ssocket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.host = 'localhost'
        self.port = 7899
        
    def start(self):
        start_time = time.time()
        print(f'{self.id} started')
        self.ssocket.bind((self.host, self.port))
        self.ssocket.listen(5) # queue up to 5 requests
        
        try:
            while True:
                # establish a connection
                clientsocket, addr = self.ssocket.accept()      
                print(f'Got a connection from {addr}')
                currentTime = time.ctime(time.time()) + "\r\n"
                clientsocket.send(currentTime.encode('ascii'))
                clientsocket.close()                  
        except KeyboardInterrupt:
            return self.terminate()
        
        
            
    def terminate(self):
        print(f'{self.id} terminated')
        self.ssocket.close()
        

In [ ]:
s = Server()
s.start()

In [ ]:
s.host

In [ ]:
import asyncore

class Handler(asyncore.dispatcher):
    def __init__(self, sock):
        self.buffer = b''
        super().__init__(sock)

    def handle_read(self):
        self.buffer += self.recv(1024)
        print(f'current buffer: {repr(self.buffer)}')


class Server(asyncore.dispatcher):
    def __init__(self, host, port):
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.set_reuse_addr()
        self.bind((host, port))
        self.listen(5)

    def handle_accepted(self, sock, addr):
        print(f'Incoming connection from {repr(addr)}')
        Handler(sock)


if __name__ == "__main__":
    server = Server("localhost", 1234)
    asyncore.loop()

In [ ]:
import asyncore, time

class Client(asyncore.dispatcher):

    def __init__(self, host, port):
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.connect( (host, port) )
        self.buffer = bytes("hello world", 'ascii')
        self.last_sent = time.time()

    def handle_connect(self):
        pass

    def handle_close(self):
        self.close()

    def handle_read(self):
        print('Re: ', self.recv(1024))

    def writable(self):
        return (len(self.buffer) > 0)

    def writable(self):
        return True

    def handle_write(self):
        sent = self.send(self.buffer)
        if sent:
            print('Sent:', sent)
            self.buffer = self.buffer[sent:]
            self.last_sent = time.time()
        else:
            if time.time() - self.last_sent > 10:
                self.buffer = 'beat'.encode('utf-8')


client = Client('localhost', 1234)
asyncore.loop()

In [ ]:
from pyensembl import EnsemblRelease
#https://github.com/openvax/pyensembl

#release 77 uses human reference genome GRCh38
#release 75 uses human reference genome GRCh37
data = EnsemblRelease(77)


def sequence_from_coordinates(chromosome, strand, start, end, ref_genome=37):
    '''
    Download the nucleotide sequence from the gene_name.
    
    Params:
            strand: "1" for the plus strand and "2" for the minus strand.
            ref_genome: 37 or 38 only (GRCh number)
    
    '''
    Entrez.email = "a.marcozzi@umcutrecht.nl" # Always tell NCBI who you are
    
    if int(ref_genome) == 37:
        #GRCh37 from http://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.25/#/def_asm_Primary_Assembly
        NCBI_IDS = {'1':'NC_000001.10','2':'NC_000002.11','3':'NC_000003.11','4':'NC_000004.11',
                    '5':'NC_000005.9','6':'NC_000006.11','7':'NC_000007.13','8':'NC_000008.10',
                    '9':'NC_000009.11','10':'NC_000010.10','11':'NC_000011.9','12':'NC_000012.11',
                    '13':'NC_000013.10','14':'NC_000014.8','15':'NC_000015.9','16':'NC_000016.9',
                    '17':'NC_000017.10','18':'NC_000018.9','19':'NC_000019.9','20':'NC_000020.10',
                    '21':'NC_000021.8','22':'NC_000022.10','X':'NC_000023.10','Y':'NC_000024.9'}
    elif int(ref_genome) == 38:
        #GRCh38 from https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.38
        NCBI_IDS = {'1':'NC_000001.11','2':'NC_000002.12','3':'NC_000003.12','4':'NC_000004.12',
                    '5':'NC_000005.10','6':'NC_000006.12','7':'NC_000007.14','8':'NC_000008.11',
                    '9':'NC_000009.12','10':'NC_000010.11','11':'NC_000011.10','12':'NC_000012.12',
                    '13':'NC_000013.11','14':'NC_000014.9','15':'NC_000015.10','16':'NC_000016.10',
                    '17':'NC_000017.11','18':'NC_000018.10','19':'NC_000019.10','20':'NC_000020.11',
                    '21':'NC_000021.9','22':'NC_000022.11','X':'NC_000023.11','Y':'NC_000024.10'}
        
  
    try:        
        handle = Entrez.efetch(db="nucleotide", 
                               id=NCBI_IDS[str(chromosome)], 
                               rettype="fasta", 
                               strand=strand, #"1" for the plus strand and "2" for the minus strand.
                               seq_start=start,
                               seq_stop=end)
        record = SeqIO.read(handle, "fasta")
        handle.close()
        sequence = str(record.seq)
        return sequence
    except ValueError:
        print('ValueError: no sequence found in NCBI')
        return False

In [ ]:
genes = ['SPEN','ANKRD36B','BRAF','AGAP3','SUMF2','EMC1','CHAC1','STK39','GPRC5C','E2F4','CAMK4','ZNF304']
genes_seq = {}
for gene_name in genes:
    d = data.genes_by_name(gene_name)[0]
    s = sequence_from_coordinates(chromosome=d.contig,
                                  strand=1,
                                  start=d.start,
                                  end=d.end,
                                  ref_genome=38)
    genes_seq[gene_name] = s 
print('done')

In [ ]:
for k,v in genes_seq.items():
    print(f'\n>{k}\n{v}')

In [ ]:
data.genes_by_name('ANKRD36B')

In [ ]:
data.genes_by_name('SPEN')

In [ ]:
a= {u'1009': {u'Avg': 188,
      u'amount_spent': 2820,
      u'size': 15,
      u'users': {u'0': 346041,
       u'1': 658546,
       u'2': 658194,
       u'3': 658316
       }},
    u'1056': {u'Avg': 140,
      u'amount_spent': 140,
      u'size': 1,
      u'users': {u'0': 645767}},
     u'1103': {u'Avg': 95,
      u'amount_spent': 285,
      u'size': 3,
      u'users': {u'0': 339285, u'1': 658440}}}
b = {u'1009': {u'Avg': 246.25,
  u'amount_spent': 1970,
  u'size': 8,
  u'users': {u'0': 61496,
   u'1': 1350,
   u'2': 164852,
   u'3': 517760,
   u'4': 576458,
   u'5': 203840,
   u'6': 52239,
   u'7': 516672}},
 u'1056': {u'Avg': 170,
  u'amount_spent': 680,
  u'size': 4,
  u'users': {u'0': 503737, u'1': 624418, u'2': 642471}}}

In [ ]:
def merge_dict(dictA, dictB):
    '''(dict, dict) => dict
    Merge two dicts, if they contain the same keys, it sums their values.
    Return the merged dict.
    
    Example:
        dictA = {'any key':1, 'point':{'x':2, 'y':3}, 'something':'aaaa'}
        dictB = {'any key':1, 'point':{'x':2, 'y':3, 'z':0, 'even more nested':{'w':99}}, 'extra':8}
        merge_dict(dictA, dictB)
        
        {'any key': 2,
         'point': {'x': 4, 'y': 6, 'z': 0, 'even more nested': {'w': 99}},
         'something': 'aaaa',
         'extra': 8}
    '''
    r = {}
    
    common_k = [k for k in dictA if k in dictB]
    common_k += [k for k in dictB if k in dictA]
    common_k = set(common_k)
    
    for k, v in dictA.items():
        #add unique k of dictA
        if k not in common_k:
            r[k] = v
        
        else:
            #add inner keys if they are not containing other dicts 
            if type(v) is not dict:
                if k in dictB:
                    r[k] = v + dictB[k]
            else:
                #recursively merge the inner dicts
                r[k] = merge_dict(dictA[k], dictB[k])
    
    #add unique k of dictB
    for k, v in dictB.items():
        if k not in common_k:
            r[k] = v

    return r
            

In [ ]:
merge_dict(a,b)

In [ ]:
dictA = {'any key':1, 'point':{'x':2, 'y':3}, 'something':'aaaa'}
dictB = {'any key':1, 'point':{'x':2, 'y':3, 'z':0, 'even more nested':{'w':99}}, 'extra':8}
merge_dict(dictA, dictB)

In [ ]:
a.update(b)
a

In [21]:


def heikin_ashi(O, H, L, C, oldO, oldC):
    '''
    Generates a Heikin-Ashi candle from ticker data.
    '''
    HA_Close = (O + H + L + C)/4
    HA_Open = (oldO + oldC)/2
    elements = np.array([H, L, HA_Open, HA_Close])
    HA_High = elements.max(0) 
    HA_Low = elements.min(0)   
    return np.array([HA_Open, HA_High, HA_Low, HA_Close])


ImmediateDeprecationError: 
Yahoo Actions has been immediately deprecated due to large breaks in the API without the
introduction of a stable replacement. Pull Requests to re-enable these data
connectors are welcome.

See https://github.com/pydata/pandas-datareader/issues


In [18]:
data.head()

""


In [ ]:
!pip install fix_yahoo_finance